In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import re as re
import os.path

import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit


# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.callbacks import History,LearningRateScheduler
# from tensorflow.keras.layers import Dropout
# print('TensorFlow %s, Keras %s, numpy %s, pandas %s'%(tf.__version__,keras.__version__, np.__version__,pd.__version__))
__DEBUG__=False

In [4]:
train = pd.read_csv('data_concatener.csv', sep=';')
test  = pd.read_csv('test_set_values.csv')
full_data = [train, test]
finalfile_index=test.id #Index des données de test pour le résultat final

# Useless column
train.drop('num_private', axis=1, inplace=True)
train.drop('date_recorded', axis=1, inplace=True)
train.drop('funder', axis=1, inplace=True)
train.drop('installer', axis=1, inplace=True)
train.drop('scheme_name', axis=1, inplace=True)
train.drop('scheme_management', axis=1, inplace=True)
train.drop('recorded_by', axis=1, inplace=True)
train.drop('wpt_name', axis=1, inplace=True)
train.drop('subvillage', axis=1, inplace=True)
test.drop('num_private', axis=1, inplace=True)
test.drop('date_recorded', axis=1, inplace=True)
test.drop('funder', axis=1, inplace=True)
test.drop('installer', axis=1, inplace=True)
test.drop('scheme_name', axis=1, inplace=True)
test.drop('scheme_management', axis=1, inplace=True)
test.drop('recorded_by', axis=1, inplace=True)
test.drop('subvillage', axis=1, inplace=True)
test.drop('wpt_name', axis=1, inplace=True)

# Double column
train.drop('quantity', axis=1, inplace=True)
train.drop('source', axis=1, inplace=True)
train.drop('waterpoint_type', axis=1, inplace=True)
train.drop('payment', axis=1, inplace=True)
train.drop('management', axis=1, inplace=True)
train.drop('extraction_type', axis=1, inplace=True)
train.drop('extraction_type_group', axis=1, inplace=True)
train.drop('water_quality', axis=1, inplace=True)
train.drop('source_type', axis=1, inplace=True)
train.drop('ward', axis=1, inplace=True)
train.drop('lga', axis=1, inplace=True)
train.drop('region', axis=1, inplace=True)
train.drop('basin', axis=1, inplace=True)
train.drop('longitude', axis=1, inplace=True)
train.drop('latitude', axis=1, inplace=True)
test.drop('quantity', axis=1, inplace=True)
test.drop('source', axis=1, inplace=True)
test.drop('waterpoint_type', axis=1, inplace=True)
test.drop('payment', axis=1, inplace=True)
test.drop('management', axis=1, inplace=True)
test.drop('extraction_type', axis=1, inplace=True)
test.drop('extraction_type_group', axis=1, inplace=True)
test.drop('water_quality', axis=1, inplace=True)
test.drop('source_type', axis=1, inplace=True)
test.drop('ward', axis=1, inplace=True)
test.drop('lga', axis=1, inplace=True)
test.drop('region', axis=1, inplace=True)
test.drop('basin', axis=1, inplace=True)
test.drop('longitude', axis=1, inplace=True)
test.drop('latitude', axis=1, inplace=True)

# Refactoring value empty
for dataset in full_data:
    dataset.loc[dataset.public_meeting.isnull(), 'public_meeting'] = 0
    dataset.loc[dataset.public_meeting == False, 'public_meeting'] = 0
    dataset.loc[dataset.public_meeting == True, 'public_meeting'] = 1
    # dataset.loc[dataset.latitude == 0.0, 'latitude'] = 35.225766
    # dataset.loc[dataset.longitude == '-2,00E-08', 'longitude'] = -6.145943
    dataset.loc[dataset.extraction_type_class == 'gravity', 'extraction_type_class'] = 0
    dataset.loc[dataset.extraction_type_class == 'submersible', 'extraction_type_class'] = 1
    dataset.loc[dataset.extraction_type_class == 'handpump', 'extraction_type_class'] = 2
    dataset.loc[dataset.extraction_type_class == 'other', 'extraction_type_class'] = 3
    dataset.loc[dataset.extraction_type_class == 'motorpump', 'extraction_type_class'] = 4
    dataset.loc[dataset.extraction_type_class == 'wind-powered', 'extraction_type_class'] = 5
    dataset.loc[dataset.extraction_type_class == 'rope pump', 'extraction_type_class'] = 6
    dataset.loc[dataset.management_group == 'user-group', 'management_group'] = 0
    dataset.loc[dataset.management_group == 'unknown', 'management_group'] = 1
    dataset.loc[dataset.management_group == 'other', 'management_group'] = 1
    dataset.loc[dataset.management_group == 'commercial', 'management_group'] = 2
    dataset.loc[dataset.management_group == 'parastatal', 'management_group'] = 3
    dataset.loc[dataset.payment_type == 'annually', 'payment_type'] = 0
    dataset.loc[dataset.payment_type == 'never pay', 'payment_type'] = 1
    dataset.loc[dataset.payment_type == 'per bucket', 'payment_type'] = 2
    dataset.loc[dataset.payment_type == 'unknown', 'payment_type'] = 3
    dataset.loc[dataset.payment_type == 'other', 'payment_type'] = 3
    dataset.loc[dataset.payment_type == 'on failure', 'payment_type'] = 4
    dataset.loc[dataset.payment_type == 'never pay', 'payment_type'] = 5
    dataset.loc[dataset.payment_type == 'monthly', 'payment_type'] = 6
    dataset.loc[dataset.quality_group == 'good', 'quality_group'] = 0
    dataset.loc[dataset.quality_group == 'salty', 'quality_group'] = 1
    dataset.loc[dataset.quality_group == 'milky', 'quality_group'] = 2
    dataset.loc[dataset.quality_group == 'fluoride', 'quality_group'] = 3
    dataset.loc[dataset.quality_group == 'colored', 'quality_group'] = 4
    dataset.loc[dataset.quality_group == 'unknown', 'quality_group'] = 5
    dataset.loc[dataset.quantity_group == 'enough', 'quantity_group'] = 0
    dataset.loc[dataset.quantity_group == 'insufficient', 'quantity_group'] = 1
    dataset.loc[dataset.quantity_group == 'dry', 'quantity_group'] = 2
    dataset.loc[dataset.quantity_group == 'seasonal', 'quantity_group'] = 3
    dataset.loc[dataset.quantity_group == 'unknown', 'quantity_group'] = 4
    dataset.loc[dataset.source_class == 'groundwater', 'source_class'] = 0
    dataset.loc[dataset.source_class == 'surface', 'source_class'] = 1
    dataset.loc[dataset.source_class == 'unknown', 'source_class'] = 2
    dataset.loc[dataset.waterpoint_type_group == 'communal standpipe', 'waterpoint_type_group'] = 0
    dataset.loc[dataset.waterpoint_type_group == 'hand pump', 'waterpoint_type_group'] = 1
    dataset.loc[dataset.waterpoint_type_group == 'other', 'waterpoint_type_group'] = 2
    dataset.loc[dataset.waterpoint_type_group == 'improved spring', 'waterpoint_type_group'] = 3
    dataset.loc[dataset.waterpoint_type_group == 'cattle trough', 'waterpoint_type_group'] = 4
    dataset.loc[dataset.waterpoint_type_group == 'dam', 'waterpoint_type_group'] = 5
    dataset.loc[dataset.permit.isnull(), 'permit'] = 0
    dataset.loc[dataset.permit == False, 'permit'] = 0
    dataset.loc[dataset.permit == True, 'permit'] = 1
    dataset.loc[dataset.construction_year == 0, 'construction_year'] = np.NaN
    construction_year_avg = dataset['construction_year'].mean() # Calcul de la valeur moyenne
    construction_year_std = dataset['construction_year'].std()  # Calcul de l'écart type
    construction_year_null_count = dataset['construction_year'].isnull().sum() # nombre de valeur nulle
    construction_year_null_random_list = np.random.randint(construction_year_avg - construction_year_std, construction_year_avg + construction_year_std, size=construction_year_null_count)   
    dataset.loc[np.isnan(dataset['construction_year']),'construction_year'] = construction_year_null_random_list    
    dataset['construction_year'] = dataset['construction_year'].astype(int)
    
    dataset.loc[dataset.population == 0, 'population'] = np.NaN
    population_avg = dataset['population'].mean() # Calcul de la valeur moyenne
    population_std = dataset['population'].std()  # Calcul de l'écart type
    population_null_count = dataset['population'].isnull().sum() # nombre de valeur nulle
    population_null_random_list = np.random.randint(population_avg, population_avg + population_std, size=population_null_count)   
    dataset.loc[np.isnan(dataset['population']),'population'] = population_null_random_list    
    dataset['population'] = dataset['population'].astype(int)
    # dataset.loc[dataset.management_group == 'unknown', 'management_group'] = 'other'
    # dataset.loc[dataset.payment_type == 'unknown', 'payment_type'] = 'other'


train['functional']= (train['status_group']=='functional')*1
train['non_functional']= (train['status_group']=='non functional')*1
train['functional_needs_repair']= (train['status_group']=='functional needs repair')*1

train.drop('status_group', axis=1, inplace=True)

BASE_DIR = os.path.abspath('')
# print(BASE_DIR)
train.to_csv(os.path.join(BASE_DIR, 'version_final.csv'))

In [5]:
print (train[["quality_group", "functional"]].groupby(['quality_group'], as_index=1).mean())
print (train[["quality_group", "non_functional"]].groupby(['quality_group'], as_index=1).mean())
print (train[["quality_group", "functional_needs_repair"]].groupby(['quality_group'], as_index=1).mean())

print('------------------------------------------------')

print (train[["quantity_group", "functional"]].groupby(['quantity_group'], as_index=1).mean())
print (train[["quantity_group", "non_functional"]].groupby(['quantity_group'], as_index=1).mean())
print (train[["quantity_group", "functional_needs_repair"]].groupby(['quantity_group'], as_index=1).mean())

print('------------------------------------------------')

print (train[["waterpoint_type_group", "functional"]].groupby(['waterpoint_type_group'], as_index=1).mean())
print (train[["waterpoint_type_group", "non_functional"]].groupby(['waterpoint_type_group'], as_index=1).mean())
print (train[["waterpoint_type_group", "functional_needs_repair"]].groupby(['waterpoint_type_group'], as_index=1).mean())

print('------------------------------------------------')

print (train[["management_group", "functional"]].groupby(['management_group'], as_index=1).mean())
print (train[["management_group", "non_functional"]].groupby(['management_group'], as_index=1).mean())
print (train[["management_group", "functional_needs_repair"]].groupby(['management_group'], as_index=1).mean())

print('------------------------------------------------')

print (train[["payment_type", "functional"]].groupby(['payment_type'], as_index=1).mean())
print (train[["payment_type", "non_functional"]].groupby(['payment_type'], as_index=1).mean())
print (train[["payment_type", "functional_needs_repair"]].groupby(['payment_type'], as_index=1).mean())

print('------------------------------------------------')

# print (train[["region", "functional"]].groupby(['region'], as_index=1).mean())
# print (train[["region", "non_functional"]].groupby(['region'], as_index=1).mean())
# print (train[["region", "functional_needs_repair"]].groupby(['region'], as_index=1).mean())


# print (train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean())

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split

# rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42)
# rnd_clf.fit(X_train, y_train)

               functional
quality_group            
0                0.565941
1                0.460828
2                0.544776
3                0.723502
4                0.502041
5                0.140725
               non_functional
quality_group                
0                    0.357236
1                    0.482002
2                    0.437811
3                    0.216590
4                    0.387755
5                    0.840618
               functional_needs_repair
quality_group                         
0                             0.076823
1                             0.057170
2                             0.017413
3                             0.059908
4                             0.110204
5                             0.018657
------------------------------------------------
                functional
quantity_group            
0                 0.652323
1                 0.523234
2                 0.025136
3                 0.574074
4                 0.269962
  

In [6]:
from sklearn.model_selection import train_test_split
#Séparation des valeurs de train et label (tous les exemples)
X_alltrain = train.values[:, 1:]
y_alltrain = train.values[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X_alltrain, y_alltrain, random_state=42)
print('%i X_train, %i X_test, %i y_train, %i y_test'%(
    X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0]))
print('%i X_alltrain, %i y_alltrain'%(X_alltrain.shape[0], y_alltrain.shape[0]))
feature_names=train.columns.tolist()[1:-4]
target_names=["Fonctionnel","Non fonctionnel","Necessite réparation"]
print('features:',feature_names)
print('target:',target_names)

44550 X_train, 14850 X_test, 44550 y_train, 14850 y_test
59400 X_alltrain, 59400 y_alltrain
features: ['amount_tsh', 'gps_height', 'region_code', 'district_code', 'population', 'public_meeting', 'permit', 'construction_year', 'extraction_type_class', 'management_group', 'payment_type', 'quality_group', 'quantity_group', 'source_class']
target: ['Fonctionnel', 'Non fonctionnel', 'Necessite réparation']


In [ ]:
"""
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
def plot_decision_boundary(clf,X,y, axes=[-0, 5, 0, 550], axis_name=['x1','x2'],alpha=0.5, contour=True):
"""
    """
    Fonction pour l'affichage 2 D des résultats   
    
    Parameters
    ----------
    clf : Classifier à afficher
    X : features de Données a afficher
    y : labels de Données a afficher  
    axes : : Tailles des axes (valeur min/max)
    axis_name : Nom des axes sur le graphique
    alpha : Transparence des points
    contour : Afichage du contour
    """     
    """
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if contour:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bo",label="Disparu", alpha=alpha)
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "ys", label="Rescapé",alpha=alpha)
    plt.axis(axes)
    plt.xlabel(axis_name[0], fontsize=18)
    plt.ylabel(axis_name[1]+ "  ",fontsize=18, rotation=0)    
    plt.legend(loc="lower right", fontsize=14)

import os
from IPython.display import Image

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import accuracy_score

tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X_train, y_train)

export_graphviz(
        tree_clf,
        out_file="train_tree.dot",
        feature_names=feature_names,
        class_names=target_names,
        rounded=True,
        filled=True
    )

os.system("dot -Tpng train_tree.dot -o train_tree.png") 
Image("train_tree.png")
"""

In [8]:
# Algorithm à utiliser : XGBClassifier to Random Forest
from sklearn.ensemble import RandomForestClassifier
# Source : http://www.philipkalinda.com/ds6.html
new_rf_model = RandomForestClassifier(random_state = 123, n_estimators = 100)
new_rf_model.fit(X_alltrain, feature_names)
predictions = new_rf_model.predict(X_alltrain)
print(len(new_targets_status[feature_names==predictions])/len(feature_names))

kf = KFold(X_alltrain.shape[0], n_folds=10,shuffle=True,random_state=123)
cvs = cross_val_score(new_rf_model,X_alltrain, feature_names,cv=kf)

def iterate_rf(iterations,X_train,Y_train,X_final):
    kf = KFold(X_train.shape[0], n_folds=10,shuffle=True,random_state=123)
    pred_df = pd.DataFrame()
    accuracies = []
    for i in range(iterations):
        rf_model = RandomForestClassifier(n_estimators = 100)
        xtrain, xtest, ytrain, ytest = train_test_split(X_train, Y_train, test_size=0.25, random_state=123)
        rf_model.fit(xtrain,ytrain)
        predictions = rf_model.predict(xtest)
        acc = len(ytest[ytest==predictions])/len(ytest)
        accuracies.append(acc)
        final_predictions = rf_model.predict(X_final)
        pred_df[i] = final_predictions
    return pred_df, accuracies

rf_df, rf_accuracies = iterate_rf(200,X_alltrain, feature_names,y_alltrain)

ValueError: Number of labels=14 does not match number of samples=59400